In [13]:
import ta
import sqlalchemy
import pandas as pd
import yfinance as yf

In [2]:
engine = sqlalchemy.create_engine('sqlite:///USA')

In [5]:
names = pd.read_sql("select name from sqlite_master where type = 'table';", con=engine)
names = names.name.to_list()
names

['MMM',
 'AXP',
 'AMGN',
 'AMZN',
 'AAPL',
 'BA',
 'CAT',
 'CVX',
 'CSCO',
 'KO',
 'DIS',
 'DOW',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'MCD',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'CRM',
 'TRV',
 'UNH',
 'VZ',
 'V',
 'WMT']

In [6]:
framelist = []

for name in names:
    framelist.append(
        pd.read_sql(f"select Date, Close from '{name}'", con=engine)
    )

In [8]:
def MACDdecision(df):
    df['MACD_diff'] = ta.trend.macd_diff(df.Close)
    df.loc[(df.MACD_diff > 0) & (df.MACD_diff.shift(1) < 0), 'Decision MACD'] = 'Buy'


def RSI_SMAdecision(df):
    df['RSI'] = ta.momentum.rsi(df.Close, window=10)
    df['SMA200'] = ta.trend.sma_indicator(df.Close, window=200)
    df.loc[(df.Close > df.SMA200) & (df.RSI < 30), 'Decision RSI/SMA'] = 'Buy'



In [9]:
for frame in framelist:
    MACDdecision(frame)
    RSI_SMAdecision(frame)

In [12]:
for name, frame in zip(names, framelist):
    if frame['Decision MACD'].iloc[-1] == 'Buy':
        print(f"Buying Signal MACD for {name}")

    if frame['Decision RSI/SMA'].iloc[-1] == 'Buy':
        print(f"Buying Signal RSI/SMA for {name}")

Buying Signal MACD for AMGN
